In [1]:
import numpy as np
import scipy as sp
import glob,os
%pylab inline --no-import-all
plt.rc('font', family='serif', serif='Times',size=15)
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=20)
plt.rc('xtick.major', size=10)
plt.rc('ytick.major', size=10)
plt.rc('ytick', labelsize=20)
plt.rc('axes', labelsize=20)
plt.rc('figure',titlesize=25)
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['axes.linewidth'] = 2.
from astropy.io import fits
import astropy.units as u
import astropy.constants as c
import pandas as pd
import logging as log
from crispy.tools.initLogger import getLogger
log = getLogger('main')
from crispy.tools.image import Image
os.chdir('/Users/mrizzo/IFS/crispy/crispy/WFIRST/')
from params import Params
par = Params()
par.hdr

Populating the interactive namespace from numpy and matplotlib


SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT                                                                         
COMMENT ************************************************************            
COMMENT ********************** General parameters ******************            
COMMENT ************************************************************            
COMMENT                                                                         
NLENS   =                  108 / # lenslets across array                        
PITCH   =             0.000174 / Lenslet pitch (meters)                         
INTERLAC=                  2.0 / Interlacing                                    
PHILENS =    26.565051177077

# Extracting the spectrum from all the microspectra in the PSF at once

I have the intuition that this improves over extracting from all the microspectra separately and then matched-filtering, although this still remains to be proven.

In this notebook, the intention is to implement the new fitting routine, and compare it with the old one.

Question: in measuring the spectrum, one usually shows the albedo (divide by the star). Should I do that when comparing the SNR?

## Construct the individual PSFs

We can do that by propagating individual slices of an off-axis PSF one at a time. This is actually a natural byproduct of the IFS propagator, since it propagates one wavelength at a time and then saves it. We can thus simply tell it to export that intermediate cube.

In [2]:
offaxis_psf_filename='/Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits'
folder = '/Users/mrizzo/IFS/Extraction/'
try:
    os.makedirs(folder)
except OSError:
    log.error("Couldn't create master folder")
    pass

from crispy.tools.postprocessing import process_offaxis
import glob
filelist = glob.glob('/Users/mrizzo/IFS/OS5/with_lowfc/*.fits')
reffiles = filelist[:30]
fileshape = fits.open(reffiles[0])[0].data.shape
BW=0.18
lamc=660
lamlist = lamc*np.linspace(1.-BW/2.,1.+BW/2.,fileshape[0])*u.nm

main - ERROR - Couldn't create master folder


In [3]:
process_offaxis(par,
                offaxis_psf_filename=offaxis_psf_filename,
                fileshape=fileshape,
                lamlist=lamlist,lamc=lamc,
                outdir_average=folder,
                Nave=1,inttime=1,Nreads=1,
                filename=par.codeRoot+'/Inputs/'+'Jupiter_1x_5AU_90deg.dat',
                planet_radius = 1.27, # in R_jup
                planet_AU = 3.6,planet_dist_pc=14.1,
                albedo=0.28,
                target_star_T=5887*u.K, target_star_Vmag=5.03,
                ref_star_T=9377*u.K, ref_star_Vmag=2.37,
                tel_pupil_area=3.650265060424805*u.m**2, order=3)

crispy - INFO - Recentering off-axis cube
crispy - INFO - Read data from HDU 0 of /Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits
crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Writing data to /Users/mrizzo/IFS/Extraction//offaxiscube.fits
crispy - INFO - Constructing off-axis cube at planet separation: 4.44 lam/D (0.26 arcsec, 8.89 lenslets)
crispy - INFO - Writing data to /Users/mrizzo/IFS/Extraction//offaxiscube_shifted.fits
crispy - ERROR - Unable to write FITS file /Users/mrizzo/IFS/Extraction//offaxiscube_star_processed.fits
crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Creating Gaussian PSFLet templates


Process Consumer-3:
  File "/Users/mrizzo/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process Consumer-4:
Process Consumer-8:
Process Consumer-7:
Process Consumer-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/mrizzo/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
  File "/Users/mrizzo/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/Users/mrizzo/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process Consumer-6:
  File "/Users/mrizzo/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
    self.run()
    self.run()
    self.run()
  File "/Users/mrizzo/anaconda2/lib/python2.7/site-packages/crispy-0.3-py2.7.egg/crispy/tools/par_utils.py", line 22, in run
  Fil

KeyboardInterrupt: 

In [ ]:
PSF_cube = fits.open(par.exportDir+'/detectorFramePoly.fits')[1].data
PSF_cube_cut = PSF_cube[:,400:620,570:690]
PSF = np.zeros_like(PSF_cube_cut)
for i in range(PSF.shape[0]):
    PSF[i] = PSF_cube_cut[i]/np.sum(PSF_cube_cut[i])
print np.amax(PSF[0])
plt.figure(figsize=(10,10))
plt.imshow(PSF[0],vmax=0.01)
plt.colorbar()

In [ ]:
psflets_flat = np.reshape(PSF, (PSF.shape[0], -1))
img = fits.open(folder+'offaxis_planet_detector.fits')[1].data[400:620,570:690]
imgstar = fits.open(folder+'offaxis_star_detector.fits')[1].data[400:620,570:690]
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.colorbar()

In [ ]:
from crispy.tools.reduction import RL,calculateWaveList
spec = RL(img,psflets_flat)[0]
star = RL(imgstar,psflets_flat)[0]

In [ ]:
plt.plot(spec)

In [ ]:
plt.plot(lamlist,spec/star)
from crispy.tools.inputScene import calc_contrast
contrast=calc_contrast(wavelist=lamlist.value,distance=3.6,radius=1.27,filename=par.codeRoot+'/Inputs/'+'Jupiter_1x_5AU_90deg.dat',albedo=0.28)
plt.scatter(lamlist,contrast,color='orange')

In [ ]:

def calcSNRtotal(Nelec,subim,psflet_subarr,pixnoise=0,npix=0,Niter=10000,plot=True,indspec=4,vect=None):
    #sumpsflets = subim*Nelec+pixnoise+1e-10
    N = psflet_subarr.shape[0]
    lstsq = np.zeros((N,Niter))
    varlstsq = np.zeros((N,Niter))
    Rvect = np.zeros((N,Niter))
    RLvals = np.zeros((N,Niter))

    psflets_flat = np.reshape(psflet_subarr, (psflet_subarr.shape[0], -1))
    A = psflets_flat.T

    for j in range(Niter):
        # subtract mean of background (assumed known)
        if j%(max(Niter/10,1))==0: print j,"/",Niter
#         Nit = np.amax((subim*Nelec+pixnoise)/0.1).astype(int)
#         if j%(max(Niter/10,1))==0: print j,"/",Niter,", Nit=",Nit

#         img = np.zeros_like(subim)
#         var = np.zeros_like(subim)
#         for h in range(Nit):
#             frame = np.random.poisson((subim*Nelec+pixnoise)/Nit.astype(float))
#             img+= frame
#             var += frame**2
#         var /= Nit.astype(float)
#         var -= (img/Nit.astype(float))**2
#         var*=Nit
#         img -= pixnoise
        img = np.random.poisson(subim*Nelec+pixnoise)
    ### WATCH OUT, USING NELEC HERE IS CHEATING ###
        sumpix = np.sum(img-pixnoise)
        guess = np.ones(N)*sumpix/float(N)
        
        rl = RL(img,psflets=psflet_subarr,prior=pixnoise,guess=guess,niter=30)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
        RLvals[:,j] = rl
        variance = np.reshape(np.sum(psflet_subarr*rl[:,np.newaxis,np.newaxis],axis=0)+pixnoise,-1)
#         variance = np.reshape(img,-1)
        img -= pixnoise

#         variance = np.reshape(var,-1)
#         variance = np.reshape(img,-1)
#         img-=pixnoise
        Ninv = np.diag(1./(variance+1e-10))
        Cinv = np.dot(np.transpose(A),np.dot(Ninv,A))
        C = np.linalg.inv(Cinv)
        Q = sp.linalg.sqrtm(Cinv)
        s = np.sum(Q,axis=0)
        R = Q/s[:,np.newaxis]
        #Ctilde = np.diag(1./(s**2+1e-10)
        varlstsq[:,j] = 1./(s**2+1e-10)
        x = np.reshape(img,-1)
        right = np.dot(np.transpose(A),np.dot(Ninv,x))
        f = np.dot(C,right)
#         rl = RL(img,psflets=psflet_subarr)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
#         RLvals[:,j] = rl
        
#         lstsq[:,j] = np.dot(C,right)#np.dot(R,f)
        lstsq[:,j] = np.dot(R,f)
        #RLvals[:,j] = RL(img+pixnoise,psflets=psflet_subarr,prior=pixnoise,guess=lstsq[:,j],niter=10)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])

#         RLvals[:,j] = np.dot(R,RLvals[:,j])
        if vect is not None: Rvect[:,j] = np.dot(R,vect)


    estSNRlstsq = np.mean(lstsq,axis=1)/np.std(lstsq,axis=1)
    estSNRRL = np.mean(RLvals,axis=1)/np.std(RLvals,axis=1)
#     estSNROptExt = np.mean(0.5*(optext[13,:]+optext[14,:]))/np.std(0.5*(optext[13,:]+optext[14,:]))
    if plot:
        plt.figure(figsize=(10,10))
        plt.imshow(img)
        plt.colorbar()
        plt.figure(figsize=(15,10))
        plt.subplot(121)
        res = img-np.reshape(np.dot(A,lstsq[:,j]),subim.shape)
        plt.imshow(res)
        Npix = np.prod(res.shape)
        plt.title('Residuals with lstsq algorithm: $\chi^2$=%.2f'%np.sum(res**2/np.reshape(np.dot(A,lstsq[:,j])+pixnoise,subim.shape)/Npix))

        plt.colorbar(fraction=0.046, pad=0.04)
        plt.subplot(122)
        res = img-np.reshape(np.dot(A,RLvals[:,j]),subim.shape)
        plt.imshow(res)
        plt.title('Residuals with EM algorithm: $\chi^2$=%.2f'%np.sum(res**2/np.reshape(np.dot(A,RLvals[:,j])+pixnoise,subim.shape)/Npix))
        plt.colorbar(fraction=0.046, pad=0.04)
        plt.figure(figsize=(15,10))
#         plt.subplot(121)
#         lamlist,_ = calculateWaveList(par,method='lstsq')
#         plt.plot(lstsq[:,-1])
#         plt.plot(RLvals[:,-1])
#         scale = np.mean(RLvals[:,-1])/np.mean(optext[4:-4,-1])
#         plt.plot(lamlist,scale*optext[3:-3,-1])
#         print np.mean(RLvals[:,-1]),np.mean(optext[4:-4,-1])

        SNR = Nelec/(np.sqrt(Nelec+npix*pixnoise))
        print "Expected SNR:",np.mean(Rvect,axis=1)[indspec]/np.sqrt(np.mean(Rvect,axis=1)[indspec]+npix*pixnoise)
        print "Estimated SNR lstsq:",estSNRlstsq[indspec]
        print "Estimated SNR RL:",estSNRRL[indspec]
        if vect is not None:
            varray = np.array(vect)+1e-10
            fchi2 = Rvect[:,-1]
#             plt.subplot(122)
            plt.plot(fchi2,label='Input')
#             plt.plot(lstsq[:,-1])
            plt.errorbar(np.arange(N),lstsq[:,-1],yerr=np.sqrt(varlstsq[:,-1]),label='last lstsq trial')
            plt.plot(np.arange(N),np.mean(lstsq,axis=1),'o',label='lstsq samp. mean')
            plt.fill_between(np.arange(N), np.mean(lstsq,axis=1)-np.std(lstsq,axis=1), np.mean(lstsq,axis=1)+np.std(lstsq,axis=1),alpha=0.3,facecolor='Gray',label='lstsq samp. 1sigma')
            plt.errorbar(np.arange(N),np.mean(RLvals,axis=1),yerr=np.std(RLvals,axis=1),fmt='o',label='EM samp. mean')
            plt.plot(RLvals[:,-1],label='last EM trial')
            plt.legend()
            
#             print "MSE lstsq:",np.sum((estSNRlstsq-fchi2)**2)/np.sum(fchi2**2)
#             print "MSE RL:",np.sum((estSNRRL-fchi2)**2)/np.sum(fchi2**2)
#             print "MSE lstsq estimated:",np.mean(np.sum((lstsq-Rvect)**2,axis=0)/np.sum(Rvect**2,axis=0))
#             print "MSE RL estimated:",np.mean(np.sum((RLvals-Rvect)**2,axis=0)/np.sum(Rvect**2,axis=0))
            print "Average $\chi^2$ lstsq estimated:",np.mean(np.sum((lstsq-Rvect)**2/varlstsq,axis=0))/N
            print "Average $\chi^2$ RL estimated:",np.mean(np.sum((RLvals-Rvect)**2/varlstsq,axis=0))/N
#             print "MSRE lstsq normalized:",np.std((estSNRlstsq-varray)/varray)
#             print "MSRE RL normalized:",np.std((estSNRRL-varray)/varray)
            
        
#         print "Estimated SNR OptExt:",estSNROptExt
        if vect is not None: print "Expected mean:",np.mean(Rvect[indspec])
        print "Estimated mean lstsq:",np.mean(lstsq,axis=1)[indspec]
        print "Estimated mean RL:",np.mean(RLvals,axis=1)[indspec]
#         print pd.DataFrame(R)
#         print np.sum(R,axis=0)
#         print R[0],np.sum(R[0])
#         print "Estimated mean:",np.mean(0.5*(optext[13,:]+optext[14,:]))

    return estSNRlstsq,estSNRRL,Rvect#,estSNROptExt

In [ ]:
plt.imshow(image)
plt.colorbar()

In [ ]:
indspec=8
SNR = np.mean(rvect,axis=1)[indspec]/np.sqrt(np.mean(rvect,axis=1)[indspec]+28*100)
print SNR

In [ ]:
x = np.arange(45)
print x
print x[::2]
print x[:-1:2]
print x[1::2]

In [ ]:
# subPSF = PSF[:-1:2,:,:]+PSF[1::2,:,:]
# subPSF = (PSF[::3,:,:]+PSF[1::3,:,:]+PSF[2::3,:,:])/3.
# subPSF = PSF[:,:,:]#+PSF[1::3,:,:]+PSF[2::3,:,:])/3.
subPSF = (PSF[:-1:2,:,:]+PSF[1::2,:,:])
# newsubPSF = np.zeros((subPSF.shape[0]+1,subPSF.shape[1],subPSF.shape[2]))
# newsubPSF[:-1,:,:] = subPSF
# newsubPSF[-1,:,:] = 1.#/np.prod(PSF.shape[1:])
# subPSF = newsubPSF
subPSF = PSF[:,:,:]

v = np.ones(len(lamlist))
spec2 = spec.copy()
# spec2[:2] = 0.0
spec2[-1] = 0.0
v = spec2/np.amax(spec2)

Nelec = 500
pixnoise= 100
# vect2 = (v[::3]+v[1::3]+v[2::3])
vect2 = 0.5*(v[:-1:2]+v[1::2])*Nelec#+v[2::3]
# newvect2 = np.zeros(vect2.shape[0]+1)
# newvect2[:-1] = vect2*Nelec
# newvect2[-1] = 100
# vect2 = newvect2
vect2 = v*Nelec
image = np.sum(PSF*v[:,np.newaxis,np.newaxis],axis=0)
image[image<0]=1e-10
estSNRlstsq,estSNRRL,rvect=calcSNRtotal(Nelec=Nelec,
        subim=image,
        psflet_subarr=subPSF,
        pixnoise=pixnoise,
        npix=28,
        plot=True,
        Niter=100,
        indspec=8,
        vect = vect2)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(estSNRlstsq)
plt.colorbar()

In [ ]:

def calcSNREM(Nelec,subim,psflet_subarr,pixnoise=0,npix=0,Niter=10000,plot=True,indspec=4,vect=None):
    #sumpsflets = subim*Nelec+pixnoise+1e-10
    N = psflet_subarr.shape[0]
    lstsq = np.zeros((N,Niter))
    varlstsq = np.zeros((N,Niter))
    Rvect = np.zeros((N,Niter))
    RLvals = np.zeros((N,Niter))

    psflets_flat = np.reshape(psflet_subarr, (psflet_subarr.shape[0], -1))
    A = psflets_flat.T
    img = np.random.poisson(subim*Nelec+pixnoise)
    img -= pixnoise
    rl = RL(img,psflets=psflet_subarr,prior=0.0)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
    RLvals[:,0] = rl
    variance = np.reshape(np.sum(psflet_subarr*rl[:,np.newaxis,np.newaxis],axis=0),-1)+pixnoise
    Ninv = np.diag(1./(variance+1e-10))
    Cinv = np.dot(np.transpose(A),np.dot(Ninv,A))
    C = np.linalg.inv(Cinv)
    Q = sp.linalg.sqrtm(Cinv)
    s = np.sum(Q,axis=0)
    R = Q/s[:,np.newaxis]
    x = np.reshape(img,-1)
    right = np.dot(np.transpose(A),np.dot(Ninv,x))
    f = np.dot(C,right)
    lstsq[:,0] = np.dot(R,f)
    
    for j in range(1,Niter):
        if j%(max(Niter/10,1))==0: print j
        # subtract mean of background (assumed known)
#         Nit = np.amax((subim*Nelec+pixnoise)/0.1).astype(int)
#         img = np.zeros_like(subim)
#         var = np.zeros_like(subim)
#         for h in range(Nit):
#             frame = np.random.poisson((subim*Nelec+pixnoise)/Nit.astype(float))
#             img+= frame
#             var += frame**2
#         var /= Nit.astype(float)
#         var -= (img/Nit.astype(float))**2
#         var*=Nit
#         img -= pixnoise
        img = np.random.poisson(subim*Nelec+pixnoise)#-pixnoise
        img-=pixnoise
        rl = RL(img,psflets=psflet_subarr,prior=0.0)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
        RLvals[:,j] = rl
        variance = np.reshape(np.sum(psflet_subarr*rl[:,np.newaxis,np.newaxis],axis=0),-1)
        Ninv = np.diag(1./(variance+1e-10))
        Cinv = np.dot(np.transpose(A),np.dot(Ninv,A))
        C = np.linalg.inv(Cinv)
        Q = sp.linalg.sqrtm(Cinv)
        s = np.sum(Q,axis=0)
        R = Q/s[:,np.newaxis]
        #Ctilde = np.diag(1./(s**2+1e-10)
        varlstsq[:,j] = 1./(s**2+1e-10)
        x = np.reshape(img,-1)
        right = np.dot(np.transpose(A),np.dot(Ninv,x))
        f = np.dot(C,right)
        
#         lstsq[:,j] = np.dot(C,right)#np.dot(R,f)
        lstsq[:,j] = np.dot(R,f)
        RLvals[:,j] = np.dot(R,RLvals[:,j])
        if vect is not None: Rvect[:,j] = np.dot(R,vect)


    estSNRlstsq = np.mean(lstsq,axis=1)/np.std(lstsq,axis=1)
    estSNRRL = np.mean(RLvals,axis=1)/np.std(RLvals,axis=1)
#     estSNROptExt = np.mean(0.5*(optext[13,:]+optext[14,:]))/np.std(0.5*(optext[13,:]+optext[14,:]))
    if plot:
        plt.figure(figsize=(10,10))
        plt.imshow(img)
        plt.colorbar()
        plt.figure(figsize=(15,10))
        plt.subplot(121)
        res = img-np.reshape(np.dot(A,lstsq[:,j]),subim.shape)
        plt.imshow(res)
        Npix = np.prod(res.shape)
        plt.title('Residuals with lstsq algorithm: $\chi^2$=%.2f'%np.sum(res**2/np.reshape(np.dot(A,lstsq[:,-1]),subim.shape)/Npix))

        plt.colorbar(fraction=0.046, pad=0.04)
        plt.subplot(122)
        res = img-np.reshape(np.dot(A,RLvals[:,0]),subim.shape)
        Npix = np.prod(res.shape)
        plt.imshow(res)
        plt.title('Residuals with EM algorithm: $\chi^2$=%.2f'%np.sum(res**2/(np.reshape(np.dot(A,RLvals[:,0]),subim.shape)+pixnoise)/Npix))
        plt.colorbar(fraction=0.046, pad=0.04)
        plt.figure(figsize=(10,10))
#         plt.subplot(121)
        lamlist,_ = calculateWaveList(par,method='lstsq')
#         plt.plot(lstsq[:,-1])
#         plt.plot(RLvals[:,-1])
#         scale = np.mean(RLvals[:,-1])/np.mean(optext[4:-4,-1])
#         plt.plot(lamlist,scale*optext[3:-3,-1])
#         print np.mean(RLvals[:,-1]),np.mean(optext[4:-4,-1])

        SNR = Nelec/(np.sqrt(Nelec+npix*pixnoise))
        print "Expected SNR:",np.mean(Rvect,axis=1)[indspec]/np.sqrt(np.mean(Rvect,axis=1)[indspec]+npix*pixnoise)
#         print "Estimated SNR lstsq:",estSNRlstsq[indspec]
        print "Estimated SNR RL:",estSNRRL[indspec]
        if vect is not None:
            varray = np.array(vect)+1e-10
            fchi2 = Rvect[:,-1]
#             plt.subplot(122)
            plt.plot(fchi2,label='Input')
            plt.plot(lstsq[:,-1])
            plt.errorbar(np.arange(N),lstsq[:,-1],yerr=np.sqrt(varlstsq[:,j]),label='lstsq')
            plt.plot(RLvals[:,-1],label='EM')
            plt.legend()
            
#             print "MSE lstsq:",np.sum((estSNRlstsq-fchi2)**2)/np.sum(fchi2**2)
#             print "MSE RL:",np.sum((estSNRRL-fchi2)**2)/np.sum(fchi2**2)
            print "MSE lstsq:",np.mean(np.sum((lstsq-Rvect)**2,axis=0)/np.sum(Rvect**2,axis=0))
            print "MSE RL:",np.mean(np.sum((RLvals-Rvect)**2,axis=0)/np.sum(Rvect**2,axis=0))
#             print "MSRE lstsq normalized:",np.std((estSNRlstsq-varray)/varray)
#             print "MSRE RL normalized:",np.std((estSNRRL-varray)/varray)
            
        
#         print "Estimated SNR OptExt:",estSNROptExt
        if vect is not None: print "Expected mean:",np.mean(Rvect[indspec])
        print "Estimated mean lstsq:",np.mean(lstsq,axis=1)[indspec]
        print "Estimated mean RL:",np.mean(RLvals,axis=1)[indspec]
        print pd.DataFrame(R)
        print np.sum(R,axis=0)
        print R[0],np.sum(R[0])
#         print "Estimated mean:",np.mean(0.5*(optext[13,:]+optext[14,:]))

    return estSNRRL#,estSNROptExt

In [ ]:
subPSF = PSF[:,:,:]#+PSF[1::3,:,:]+PSF[2::3,:,:])/3.
subPSF = (PSF[:-1:2,:,:]+PSF[1::2,:,:])
newsubPSF = np.zeros((PSF.shape[0]+1,PSF.shape[1],PSF.shape[2]))
newsubPSF[:-1,:,:] = subPSF
newsubPSF[-1,:,:] = 1.#/np.prod(PSF.shape[1:])
subPSF = newsubPSF


In [ ]:
subPSF = (PSF[:-1:2,:,:]+PSF[1::2,:,:])
# newsubPSF = np.zeros((subPSF.shape[0]+1,subPSF.shape[1],subPSF.shape[2]))
# newsubPSF[:-1,:,:] = subPSF
# newsubPSF[-1,:,:] = 1.#/np.prod(PSF.shape[1:])
# subPSF = newsubPSF
subPSF = PSF[:,:,:]

v = np.ones(len(lamlist))
spec2 = spec.copy()
# spec2[:2] = 0.0
spec2[-1] = 0.0
v = spec2/np.amax(spec2)

Nelec = 1000
pixnoise= 50.0
# vect2 = (v[::3]+v[1::3]+v[2::3])
# vect2 = (v[:-1:2]+v[1::2])*Nelec#+v[2::3]
# newvect2 = np.zeros(vect2.shape[0]+1)
# newvect2[:-1] = vect2*Nelec
# newvect2[-1] = 100
# vect2 = newvect2
vect2 = v*Nelec
image = np.sum(PSF*v[:,np.newaxis,np.newaxis],axis=0)
print np.sum(image)
image[image<0]=1e-10

img = np.random.poisson(image*Nelec+pixnoise)
plt.imshow(img)
plt.colorbar()
rl1 = RL(img,psflets=subPSF,prior=pixnoise,guess=np.ones(len(vect2))*Nelec,niter=10)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
rl2 = RL(img,psflets=subPSF,prior=pixnoise,guess=np.ones(len(vect2))*Nelec,niter=100)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
rl3 = RL(img,psflets=subPSF,prior=pixnoise,guess=np.ones(len(vect2))*Nelec,niter=1000)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
rl4 = RL(img,psflets=subPSF,prior=pixnoise,guess=np.ones(len(vect2))*Nelec,niter=2)[0]#np.dot(R,RL(img,psflets=psflet_subarr)[0])
plt.figure()
plt.plot(rl1)
plt.plot(rl2)
plt.plot(rl3)
plt.plot(rl4)
plt.plot(vect2,'o')
plt.figure()
psflets_flat = np.reshape(subPSF, (subPSF.shape[0], -1))

plt.imshow(img-np.reshape(np.dot(psflets_flat.T,rl1),img.shape)-pixnoise)
plt.colorbar()